<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd 

<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [ ]:
df = pd.read_csv("gdrive/My Drive/mbti/all_users_v2.csv", sep = ';', header = 0) 

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt

<span style="font-size:1.1em;">Verisetinin **class** tiplerine göre dağılım</span>

In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby('typeClass').type.count().plot.bar(ylim=0)
plt.savefig('dataset_distribution_typeClass.png')
plt.show()

<span style="font-size:1.1em;">Verisetindeki **I/E** dağılımını göstermektedir. I olanlar x ekseninde 0 olarak E olanlar ise 1 olarak gösterilmektedir</span>

In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby('I/E').type.count().plot.bar(ylim=0)
plt.savefig('gdrive/My Drive/mbti/dataset_distribution_I-E.png')
plt.show()

<span style="font-size:1.1em;">Verisetindeki **S/N** dağılımını göstermektedir. N olanlar x ekseninde 0 olarak S olanlar ise 1 olarak gösterilmektedir</span>

In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby('S/N').type.count().plot.bar(ylim=0)
plt.savefig('gdrive/My Drive/mbtidataset_distribution_S-N.png')
plt.show()

<span style="font-size:1.1em;">Verisetindeki **T/F** dağılımını göstermektedir. F olanlar x ekseninde 0 olarak T olanlar ise 1 olarak gösterilmektedir</span>

In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby('T/F').type.count().plot.bar(ylim=0)
plt.savefig('gdrive/My Drive/mbti/dataset_distribution_T-F.png')
plt.show()

<span style="font-size:1.1em;">Verisetindeki **J/P** dağılımını göstermektedir. P olanlar x ekseninde 0 olarak J olanlar ise 1 olarak gösterilmektedir</span>

In [ ]:
fig = plt.figure(figsize=(8,6))

df.groupby('J/P').type.count().plot.bar(ylim=0)
plt.savefig('gdrive/My Drive/mbti/dataset_distribution_J-P.png')
plt.show()

Aşağıda ise verisetinin 16 farklı MBTI kişilik tiplerine göre dağılımı gösterilmektedir.

In [ ]:
fig = plt.figure(figsize=(8,4))

df.groupby('type').type.count().plot.bar(ylim=0)
plt.savefig('gdrive/My Drive/mbti/dataset_distribution_type.png')
plt.show()

Stop words, drive'da /mbti altındaki stop_words_tr.txt okunarak alınır

---



In [ ]:
file = open("gdrive/My Drive/mbti/stop_words_tr.txt")
stop_word_list = file.read().split('\n')
file.close()

In [ ]:
stop_word_list

# Pre-Processing

Entrylere ön işleme adımları uygulanır. Bu adımlar:


1.   Bütün harflerin küçük harf haline getirilmesi

2.   '**bkz**', '**--- spoiler ---**', '**spoiler**', '**#12341324**' (gibi böyle rakamların devam ettiği) entrylerin verisetinden çıkarılması gerekmektedir.

3. Entrylerden stop words'ler silinmelidir.

4. Web sitelerinin temizlenmesi

5. Noktalama işaretlerinin temizlenmesi

6. Rakamların temizlenmesi

7. Stemming kullanılması

Temizlenme işlemlerinde empty string yerine space (' ') ile replace edilerek yapılmalıdır. Arından da fazla boşluklar vs trim edilmelidir.

```[+-]?([0-9]*)([.][0-9])?``` regex olarak kullanılacak hem rakamların silinmesinde hem de ondaklı sayıların silinmesinde --> buna gerek kalmadı


Entrylerdeki harfler küçük harf haline getirilir

In [ ]:
df['entry'] = df['entry'].str.lower()

Dataframe içerisindeki bütün entrylere '**bkz**', '**spoiler**', '**#**' içerip içermediğine bakar ve bunun sonucu index numarasıyla birlikte döner. ```15 True``` gibi bu demek oluyor ki 15 numaralı index bizim yazmış olduğumuz koşulu sağlamaktadır.

In [ ]:
indexes_contains_unwanted_words = df['entry'].str.contains('|'.join(['bkz', 'spoiler', r'#\d*']))

Dataframe'den ilgili entryler çıkartılır

In [ ]:
df = df[~indexes_contains_unwanted_words]
df.shape[0]

Entrylerden stop words silinir.   --> Bu kısım gerekmeyebilir


In [ ]:
df['entry'] = df['entry'].str.replace('|'.join(stop_word_list), ' ')

Entrylerden web site linkleri silinir.

In [ ]:
df['entry'] = df['entry'].str.replace(r'http\S+', ' ', regex=True)
df['entry'] = df['entry'].str.replace(r'www\S+', '', regex=True)

Entrylerden noktalama işaretleri temizlenir.

In [ ]:
import string

df['entry'] = df['entry'].apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))

Entrylerden rakamlar silinir.

In [ ]:
df['entry'] = df['entry'].str.replace('\d+', ' ')

Verisetinin stemmera sokmadan önceki ön işlemeden geçirilmiş hali kaydedilir. Buradaki amaç stemmerdan geçirilmiş ve geçirilmemiş halinin başarı açısından karşılaştırılmasının yapılabilmesi.

In [ ]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_v1.csv", sep=';', encoding='utf-8')

SnowballStemmer içerisinde Türkçe dili için kullanabileceğimiz TurkishStemmer import edilir. Stemmer ile kelimelerin köklerine ulaşılır. Lemmatization işlemiyle kök bulunduğu zaman elde edilen kökler biçimsel açıdan da doğru olurken (yani mantıksal açıdan da) stemmer ile elde edilen köklerde böyle bir durum söz konusu değildir. Dolayısıyla stemming edilmis hallerine bakıldıgında mantıksal açıdan düzgün sonuclar vermeyebilir.

Google Colab'a snowballstemmer ile ilgil package'i indirmek için aşağıdaki komutu çalıştırıyoruz.

In [ ]:
!pip install snowballstemmer

In [ ]:
from snowballstemmer import TurkishStemmer

In [ ]:
stemmer = TurkishStemmer()

row_count = df.shape[0]
kökler = stemmer.stemWords(entry.split(" "))

for i in range(row_count):
    entry = df['entry'][i]
    entry_kokler = stemmer.stemWords(entry.split(" "))
    df['entry'][i] = " ".join(entry_kokler)
    
df['entry']

Stemmera sokulmuş hali kaydedilir.

In [ ]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_v2.csv", sep=';', encoding='utf-8')

TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), stop_words = stop_word_list, max_features=5000, analyzer = 'word', token_pattern=r'\w{1,}')

**Feature Extraction** 

Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.

In [ ]:
tfidf_vectorizer.fit(df['entry'])

Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['entry'], df['typeClass'], random_state = 42, test_size = 0.20)

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [ ]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)